In [ ]:
import pdfplumber
# import re
import dill
from itertools import tee
import src.models
import src.helpers
import src.pdfutils

In [ ]:
filename = '/home/james/Massive/PROJECTDATA/nyc_real_estate_data/dictionaries/mapPLUTO_data_dictionary.pdf'

* Looking at the PLUTO data dictionary, it seems that most category variables are labeled as "alpahnumeric" even if they only contain numbers, such as zip codes.
* There are some exceptions, police precincts and districts are numeric and listed as such. However as there a limited number of repeating variables, I wil treat them as categorical as well.

In [ ]:
# def normalize_top(words, tolerance=0.3):
#     """Adjust 'top' values so that small variations within tolerance are treated as equal. This is necessary when parsing PDFs where words on a line may have slightly different top positions. Made with the help of ChatGPT"""
#     sorted_by_top = sorted(words, key=lambda w: w["top"])
#     clusters = []

#     for word in sorted_by_top:
#         if not clusters or abs(word["top"] - clusters[-1][0]) > tolerance:
#             clusters.append((word["top"], []))  # Create new cluster
#         clusters[-1][1].append(word)

#     # Assign the lowest top value in each cluster
#     top_mapping = {}
#     for cluster_top, cluster_words in clusters:
#         for word in cluster_words:
#             top_mapping[word["top"]] = cluster_top

#     sorted_words = sorted(words, key=lambda w: (top_mapping[w["top"]], w["x0"]))
#     return sorted_words

# def map_pdf(pdf_path, same_line_tolerance=0.3):
#     fulltext = {}
#     with pdfplumber.open(pdf_path) as pdf:
#         char_index = 0
#         all_lines = []  # Store all detected lines first
#         for page in pdf.pages:
#             last_top = None
#             words = page.extract_words()  
#             sorted_words = normalize_top(words, same_line_tolerance)
#             line = []

#             for word in sorted_words:
#                 word_length = len(word["text"])
#                 word['range'] = (char_index, char_index + word_length)
#                 char_index += word_length + 1

#                 if last_top is None or abs(word["top"] - last_top) > same_line_tolerance:  # New line
#                     line_start = word['range'][0]
#                     line_end = word['range'][1]
#                     line_range = (line_start, line_end)
#                     if last_top is not None:
#                         all_lines.append((line_range, line))  # Store completed line
#                     last_top = word["top"]
#                     line = [word]
#                 else:
#                     line.append(word)

#             if line:  # Ensure the last line is added
#                 line_end = word['range'][1]
#                 line_range = (line_start, line_end)
#                 all_lines.append((line_range, line))

#             # Store lines, skipping first and last. The first line is the title, and thus identical for each page, the last line is just the page number.
#             line_no = 0
#             for line in all_lines[1:-1]:  # Slice to remove first and last lines
#                 char_range = (next(iter(line[1]))['range'][0],  next(reversed(line[1]))['range'][1])
#                 x_range = (next(iter(line[1]))['x0'],  next(reversed(line[1]))['x1'])
#                 line_info = {'range' : char_range, 'x_dims' : x_range }
#                 line_content = ' '.join([w['text'] for w in line[1]])
#                 print(line_content)
#                 fulltext[(line_no, line_content)] = (line_info, line)
#                 line_no += 1
#     return fulltext

fulltext = src.pdfutils.map_pdf(filename)

In [ ]:
for key in fulltext.keys():
    print(key)

In [ ]:
def get_word_starts_x(line):
   starts = [word['x0'] for word in line]
   return starts


tables = {}
in_description = False
in_table = False


for key,value in fulltext.items():
    line = key[1]
    line_start = value[0]['range'][0]
    line_start = value[0]['range'][1]
    # Make sure description is set to False at the beginning of each definition, as 'Field Name' is the first part of each definition
    if line.startswith('Field Name:'):
        in_description = False
        in_table = False
        if len(value[1][1]) > 2: # Exclude the explanation of "Field Name" itself on page 3
            field_name = value[1][1][-1]['text'][1:-1] # Get the field name minus the enclosing parentheses
        table = []
        continue
    # Detect the beginning of a description section, which might contain a table (anything outside it does not contain a table I am interested in)
    if line.startswith('Description:'):
        in_description = True
        prev_line_start = line_start
    if in_description is True:
        # print('in description:', line)
        # If inside a description section, check for lines that start with value or `VALUE`, as this is what the first line of a table alway starts with.
        # As there are lines that start with `VALUE` but are not tables, also check if the line starts at a larger x0 than the previous line.
        if (line.startswith('Value') or line.startswith('VALUE')) and len(value[1][1]) <= 3: # Maximum number of words in a column heading
            print("Start of table")
            print('Table heading', line)
            col_starts = get_word_starts_x(value[1][1])
            in_table = True
            print(col_starts)
            prev_line_start = line_start
            table = [(line, value)]
        elif in_table is True and (abs(col_starts[0] - get_word_starts_x(value[1][1])[0]) < .5) :
            print('Row: ', line)
            table.append((line, value))
        elif in_table is True:
            print("End of table")
            tables[field_name] = table
            in_table = False

In [ ]:
table_dicts = {}

for field,table in tables.items():
    rows = []
    k1,k2 = table[0][0].split(' ', 1)
    for i in table[1:]:
        row = i[0].split(' ', 1)
        rows.append({k1: row[0], k2: row[1] })
    table_dicts[field] = rows



In [ ]:
for k,d in table_dicts.items():
    print(k, d)

In [ ]:

def parse_zoning(pdf_path):
    all_tables = {}
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            # Extract raw text as lines
            lines = page.extract_text().splitlines()
            # Extract tables
            tables = page.extract_tables()
            for table_index, table in enumerate(tables):
                # Find the position of the table in the raw text
                table_start_line = find_table_start(lines, table)
                # Extract the line before the table, if available
                label_line = (
                    lines[table_start_line - 2] if table_start_line > 0 else None
                )
                table = [row for row in table if "Abbreviation" not in row]
                if label_line is not None:
                    if "APPENDIX" in label_line:
                        label_line = re.sub("APPENDIX.*: ", "", label_line)
                        label_line = re.sub(" +", "_", label_line.lower())
                        prev_label_line = label_line
                    elif "PLUTO DATA DICTIONARY" in label_line:
                        label_line = None
                    elif "APPENDIX" not in label_line:
                        print("what's this?: ", print('label_line is', label_line))
                        table = [row for row in table if "Abbreviation" not in row]
                    if label_line != None:
                        all_tables[label_line] = table
                    else:
                        all_tables[prev_label_line] = all_tables[prev_label_line] + table
                else:
                    print('table_index is', table_index)
                    print('missed:', lines[table_start_line])
    return all_tables


def find_table_start(lines, table):
    """
    Identify the start of the table in the text by matching table rows
    """
    for i, line in enumerate(lines):
        # Convert the table's first row into a string and search for it in the text
        table_row = " ".join(str(cell) for cell in table[1] if cell)  # Skip empty cells
        if line in table_row:
            return i
    return -1

* Add tables from appendixes

In [ ]:
table_dicts = parse_zoning(filename)

In [ ]:
print(table_dicts)

* Parse Appendix D

In [ ]:
# with pdfplumber.open(filename) as pdf:
#     page = pdf.pages[-1]
#     # Extract raw text as lines
#     lines = page.extract_text(layout=True).splitlines()
#     words = page.extract_words(use_text_flow=False)

In [ ]:
# print(words)

In [ ]:
# import pdfplumber

# def group_words_by_row(words, y_thresh=5):
#     """Groups words into rows based on proximity of their 'top' values."""
#     words = sorted(words, key=lambda w: w['top'])  # Sort by vertical position
#     rows = []
    
#     for word in words:
#         added = False
#         for row in rows:
#             if abs(word['top'] - row[0]['top']) <= y_thresh:  # Same row if close in top values
#                 row.append(word)
#                 added = True
#                 break
#         if not added:
#             rows.append([word])  # Start a new row
    
#     return rows

# def adjust_bounding_boxes(rows):
#     """Expands row bounding boxes so all words in a row share the same top/bottom."""
#     adjusted_rows = []
    
#     for row in rows:
#         min_top = min(word['top'] for word in row)
#         max_bottom = max(word['bottom'] for word in row)
        
#         adjusted_row = []
#         for word in row:
#             adjusted_word = word.copy()
#             adjusted_word['top'] = min_top
#             adjusted_word['bottom'] = max_bottom
#             adjusted_row.append(adjusted_word)
        
#         adjusted_rows.append(adjusted_row)
    
#     return adjusted_rows

# def merge_words_into_rows(words, x_thresh=10, y_thresh=5):
#     """
#     Groups words into rows and merges horizontally close words.
    
#     Returns:
#     - List of lists, where each inner list represents a row with merged text blocks.
#     """
#     # Step 1: Group words into rows
#     rows = group_words_by_row(words, y_thresh)
    
#     # Step 2: Adjust bounding boxes for row uniformity
#     adjusted_rows = adjust_bounding_boxes(rows)
    
#     # Step 3: Merge words within each row into text segments
#     row_blocks = []
    
#     for row in adjusted_rows:
#         print('ROW IS:', row)
#         row.sort(key=lambda w: w['x0'])  # Sort words left-to-right
#         line_blocks = []
#         line = []
        
#         for word in row:
#             if line and (word['x0'] - line[-1]['x1']) <= x_thresh:  # Merge if close horizontally
#                 line.append(word)
#             else:
#                 if line:
#                     line_blocks.append(line)
#                 line = [word]
#         if line:
#             line_blocks.append(line)
        
#         # Convert words to formatted output
#         row_blocks.append([
#             (" ".join(word['text'] for word in line),  # Merged text
#              (min(line, key=lambda w: w['x0'])['x0'],  # Bounding box (x0, top, x1, bottom)
#               min(line, key=lambda w: w['top'])['top'],
#               max(line, key=lambda w: w['x1'])['x1'],
#               max(line, key=lambda w: w['bottom'])['bottom']),
#              len(line))  # Word count
#             for line in line_blocks
#         ])
    
#     return row_blocks


In [ ]:
# with pdfplumber.open(filename) as pdf:
#     page = pdf.pages[-1]
#     words = page.extract_words(use_text_flow=True)
#     text_blocks = merge_words_into_rows(words, x_thresh=10, y_thresh=20)

In [ ]:
# import pdfplumber

# def group_words_by_row(words, y_thresh=5):
#     """Groups words into rows based on proximity of their 'top' values."""
#     words = sorted(words, key=lambda w: w['top'])  # Sort by vertical position
#     rows = []
    
#     for word in words:
#         added = False
#         for row in rows:
#             if abs(word['top'] - row[0]['top']) <= y_thresh:  # Same row if close in top values
#                 row.append(word)
#                 added = True
#                 break
#         if not added:
#             rows.append([word])  # Start a new row
    
#     return rows

# def detect_header_by_uppercase(rows):
#     """Identifies the header row by checking if all words are uppercase."""
#     header_row = []
#     body_rows = []
    
#     for row in rows:
#         if all(word['text'].isupper() for word in row):  # All words must be uppercase
#             header_row = row
#             print('header_row:', [w['text'] for w in header_row])
#         else:
#             body_rows.append(row)
    
#     return header_row, body_rows

# def merge_words_into_rows(words, x_thresh=10, y_thresh=5):
#     """
#     Groups words into rows and merges horizontally close words.
    
#     - Detects headers based on uppercase text in all words.
#     - Merges words and handles row alignment.
    
#     Returns:
#     - List of lists, where each inner list represents a row with merged text blocks.
#     """
#     # Step 1: Group words into rows
#     rows = group_words_by_row(words, y_thresh)
    
#     # Step 2: Detect the header row based on all uppercase words
#     header_row, body_rows = detect_header_by_uppercase(rows)

#     # Step 3: Merge words within each row into text segments
#     all_rows = [header_row] + body_rows  # Ensure headers come first
#     row_blocks = []
    
#     for row in all_rows:
#         row.sort(key=lambda w: w['x0'])  # Sort words left-to-right
#         line_blocks = []
#         line = []
        
#         for word in row:
#             if line and (word['x0'] - line[-1]['x1']) <= x_thresh:  # Merge if close horizontally
#                 line.append(word)
#             else:
#                 if line:
#                     line_blocks.append(line)
#                 line = [word]
#         if line:
#             line_blocks.append(line)
        
#         # Convert words to formatted output
#         row_blocks.append([
#             (" ".join(word['text'] for word in line),  # Merged text
#              (min(line, key=lambda w: w['x0'])['x0'],  # Bounding box (x0, top, x1, bottom)
#               min(line, key=lambda w: w['top'])['top'],
#               max(line, key=lambda w: w['x1'])['x1'],
#               max(line, key=lambda w: w['bottom'])['bottom']),
#              len(line))  # Word count
#             for line in line_blocks
#         ])
    
#     return row_blocks


In [ ]:
# # Example Usage
# with pdfplumber.open(filename) as pdf:
#     page = pdf.pages[-1]
#     words = page.extract_words(use_text_flow=True)
#     table = merge_words_into_rows(words, x_thresh=10, y_thresh=20)

# for row in table:
#     print(row)

In [ ]:
# print(text_blocks[5])

In [ ]:
# for idx, block in enumerate(text_blocks):
#     print(idx, block)


In [ ]:
# import pdfplumber

# def group_words_by_row(words, y_thresh=5):
#     """Groups words into rows based on proximity of their 'top' values."""
#     words = sorted(words, key=lambda w: w['top'])  # Sort by vertical position
#     rows = []
    
#     for word in words:
#         print(word)
#         added = False
#         for row in rows:
#             if abs(word['top'] - row[-1]['top']) <= y_thresh:  # Compare with last word in row
#                 row.append(word)
#                 added = True
#                 break
#         if not added:
#             rows.append([word])  # Start a new row
    
#     return rows

# def detect_header_by_uppercase(rows, y_thresh=10):
#     """Identifies the header row by checking if all words are uppercase, with y-threshold filtering."""
#     header_row = []
#     body_rows = []
    
#     for row in rows:
#         if all(word['text'].isupper() for word in row):  # All words must be uppercase for header
#             if row[0]['top'] < y_thresh:  # Ensure header is within the correct vertical space
#                 header_row.extend(row)  # Combine all words of the header row
#             else:
#                 body_rows.append(row)
#         else:
#             body_rows.append(row)
    
#     return header_row, body_rows

# def merge_words_in_row(row, x_thresh=10, y_thresh=5):
#     """
#     Merges words in a single row, considering the provided x_thresh and y_thresh.
    
#     Args:
#     - row: List of word dicts in the row.
#     - x_thresh: Horizontal threshold for merging words.
#     - y_thresh: Vertical threshold for grouping words into rows.
    
#     Returns:
#     - A list of merged text blocks, each with the merged text, bounding box, and word count.
#     """
#     row.sort(key=lambda w: w['x0'])  # Sort words left-to-right
#     line_blocks = []
#     line = []
    
#     for word in row:
#         if line and (word['x0'] - line[-1]['x1']) <= x_thresh:  # Merge if close horizontally
#             line.append(word)
#         else:
#             if line:
#                 line_blocks.append(line)
#             line = [word]
#     if line:
#         line_blocks.append(line)
    
#     # Convert words to formatted output
#     return [
#         (" ".join(word['text'] for word in line),  # Merged text
#          (min(line, key=lambda w: w['x0'])['x0'],  # Bounding box (x0, top, x1, bottom)
#           min(line, key=lambda w: w['top'])['top'],
#           max(line, key=lambda w: w['x1'])['x1'],
#           max(line, key=lambda w: w['bottom'])['bottom']),
#          len(line))  # Word count
#         for line in line_blocks
#     ]

# def merge_words_into_rows(words, header_x_thresh=10, header_y_thresh=10, body_x_thresh=15, body_y_thresh=5):
#     """
#     Groups words into rows and merges horizontally close words for header and body rows with different thresholds.
    
#     - Detects headers based on uppercase text in all words.
#     - Merges words and handles row alignment with separate thresholds for header and body rows.
    
#     Returns:
#     - List of lists, where each inner list represents a row with merged text blocks.
#     """
#     # Step 1: Group words into rows
#     rows = group_words_by_row(words, body_y_thresh)  # Use body row y_thresh for initial grouping
    
#     # Step 2: Detect the header row based on all uppercase words and vertical threshold
#     header_row, body_rows = detect_header_by_uppercase(rows, y_thresh=header_y_thresh)

#     # Step 3: Merge words in header and body rows with separate thresholds
#     all_rows = [header_row] + body_rows  # Ensure headers come first
#     row_blocks = []
    
#     # Merge header row with header-specific thresholds
#     if header_row:
#         row_blocks.append(merge_words_in_row(header_row, x_thresh=header_x_thresh, y_thresh=header_y_thresh))
    
#     # Merge body rows with body-specific thresholds
#     for row in body_rows:
#         row_blocks.append(merge_words_in_row(row, x_thresh=body_x_thresh, y_thresh=body_y_thresh))
    
#     # Sort rows by top value to maintain correct vertical order
#     # Note: Sorting by both 'top' and 'x0' (for tie-breaking in case 'top' is the same) ensures correct ordering
#     final_row_blocks = []
#     for row_block in row_blocks:
#         final_row_blocks.append(sorted(row_block, key=lambda w: (w[1][1], w[1][0])))  # Sort by 'top' and 'x0'
    
#     return final_row_blocks



In [ ]:
import pdfplumber

def group_words_by_row(words, y_thresh=5):
    """Groups words into rows based on vertical proximity, allowing small deviations in top values."""
    words = sorted(words, key=lambda w: w['top'])  # Sort words top-to-bottom
    rows = []

    for word in words:
        added = False
        for row in rows:
            # Compare with first word in the row for stability
            if abs(word['top'] - row[0]['top']) <= y_thresh:
                row.append(word)
                added = True
                break
        if not added:
            rows.append([word])

    return rows

def merge_words_in_row(row, x_thresh=10):
    """
    Merges words in a single row, considering the provided x_thresh for horizontal grouping.
    
    Returns:
    - A list of merged text blocks, each with the merged text and bounding box.
    """
    row.sort(key=lambda w: w['x0'])  # Sort words left-to-right
    merged_blocks = []
    current_block = []

    for word in row:
        if current_block and (word['x0'] - current_block[-1]['x1']) <= x_thresh:
            current_block.append(word)
        else:
            if current_block:
                merged_blocks.append(current_block)
            current_block = [word]

    if current_block:
        merged_blocks.append(current_block)

    return [
        {
            "text": " ".join(w["text"] for w in block),
            "x0": min(w["x0"] for w in block),
            "x1": max(w["x1"] for w in block),
            "top": min(w["top"] for w in block),
            "bottom": max(w["bottom"] for w in block),
        }
        for block in merged_blocks
    ]

def merge_words_into_rows(words, x_thresh=10, y_thresh=5):
    """
    Groups words into rows and merges horizontally close words.
    """
    rows = group_words_by_row(words, y_thresh)
    merged_rows = [merge_words_in_row(row, x_thresh) for row in rows]
    return merged_rows

def assign_columns_to_blocks(merged_rows, column_gap_thresh=20):
    """
    Assigns a column index to each merged text block by detecting significant gaps in x0 values.
    
    Parameters:
    - merged_rows: List of lists of merged word blocks.
    - column_gap_thresh: Minimum gap to consider as a column boundary.
    
    Returns:
    - A list where each element is a tuple (column_index, word_block_dict).
    """
    all_x_values = sorted(set(block["x0"] for row in merged_rows for block in row))

    # Detect gaps to determine column boundaries
    column_boundaries = [all_x_values[0]]
    for i in range(1, len(all_x_values)):
        if all_x_values[i] - all_x_values[i - 1] > column_gap_thresh:
            column_boundaries.append(all_x_values[i])

    def get_column_index(x0):
        """Finds the appropriate column index for a given x0 value."""
        for i, boundary in enumerate(column_boundaries):
            if x0 < boundary:
                return max(i - 1, 0)
        return len(column_boundaries) - 1

    structured_output = []
    for row in merged_rows:
        row_output = [(get_column_index(block["x0"]), block) for block in row]
        structured_output.append(row_output)

    return structured_output




In [ ]:
# Example usage
header_x_thresh = 10
header_y_thresh = 1
column_gap_thresh = 20  # Adjust based on observed spacing

with pdfplumber.open(filename) as pdf:
    words = pdf.pages[-1].extract_words()  # Extract words from page 0
    merged_rows = merge_words_into_rows(words, header_x_thresh, header_y_thresh)
    structured_output = assign_columns_to_blocks(merged_rows, column_gap_thresh)

for row in structured_output:
    print(row)  # Prints the structured output

In [ ]:
header_x_thresh = 10
header_y_thresh = 10
body_x_thresh = 10
body_y_thresh = 20

# merged_rows = merge_words_into_rows(words, header_x_thresh, header_y_thresh, body_x_thresh, body_y_thresh)
merged_rows = merge_words_into_rows(words, header_x_thresh, header_y_thresh)


In [ ]:
for row in merged_rows:
    print(row)

In [ ]:
def get_pseudo_table(filename, page, ncol, top_boundary_text, bottom_boundary_text, header_x_thresh=10, header_y_thresh=10, body_x_thresh=10, body_y_thresh=20, column_gap_thresh=20):
    """This for extracting table-like arrangements of text in a PDF (which I call "pseudo-tables"), for cases where the "table" is not explicitly defined as such and thus missed by `pdfplumber.extract_tables()`
    To help correctly define the table columns, this function allows the user to provide some hints to assist in identifying boundaries. For the moment, this is only for this particular "table" (pseudo-table), but I hope it turns out to be reusable.

    Args:
        filename (_type_): Path to the pdf file
        page (_type_): Page on which the table is located
        ncol (_type_): Number of columns in the table
        top_boundary_text (_type_): Unique string of text in the last line before the start of the table header
        bottom_boundary_text (_type_): Unique string of text in the first line after the end of the table
        header_x_thresh (_type_): Horizontal distance within which words will be grouped into the same block, in the header
        header_y_thresh (_type_): Vertical distance within which words will be grouped into the same block, to account for table rows that have multiple lines of text inside them, in the header.
        body_x_thresh (_type_): Horizontal distance within which words will be grouped into the same block, in the table body
        body_y_thresh (_type_): Vertical distance within which words will be grouped into the same block, to account for table rows that have multiple lines of text inside them, in the table body.
        column_gap_thresh (_type_): Adjust based on observed spacing
    """
    column_gap_thresh = 20  # Adjust based on observed spacing

    with pdfplumber.open(filename) as pdf:
        words = pdf.pages[-1].extract_words()  # Extract words from page 0
        merged_rows = merge_words_into_rows(words, header_x_thresh, header_y_thresh)
        structured_output = assign_columns_to_blocks(merged_rows, column_gap_thresh)
    
    return structured_output





In [ ]:
ncol = 3
cols = {i:[] for i in range(ncol)}
for row in merged_rows:
    if len(row) <= ncol:
        print(row)
    for n in range(len(row)):
        coldata = [row[n] for row in merged_rows]
        cols[n] = coldata

In [ ]:
blocks[0]

In [ ]:
with open("environment_data/table_dicts.pkl", "wb") as f:
    dill.dump({'table_dicts' : table_dicts}, f)